In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
'''
多头注意力（Multi-Head Attention）机制通过并行的多个注意力头（heads）
来捕获不同子空间中的信息，每个头可以独立关注输入数据的不同方面，
从而增强模型的表示能力。
'''
class MultiHeadAttention(nn.Module):
    '''
    多头注意力
    
    - `key_size`、`query_size` 和 `value_size`：分别是键、查询和值的尺寸。
    - `num_hiddens`：隐藏层的神经元数量。
    - `num_heads`：注意力头的数量。
    - `dropout`：Dropout层的概率。
    - `W_q`、`W_k` 和 `W_v`：线性变换矩阵，用于将查询、键和值映射到多头注意力的输入空间。
    - `W_o`：线性变换矩阵，用于将多个头的输出拼接在一起后再进行变换。

    '''
    def __init__(self, 
                 key_size, 
                 query_size, 
                 value_size, 
                 num_hiddens,
                 num_heads, 
                 dropout,
                 bias=False,
                 **kwargs
                ):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    def forward(self, queries, keys, values, valid_lens):
        '''
        - `queries`、`keys` 和 `values`：输入的查询、键和值。
        - `valid_lens`：有效长度，用于掩蔽（mask）操作。
        
        1. 将查询、键和值通过线性变换，并使用 `transpose_qkv` 函数进行形状变换，
            使其适应多头注意力的计算。

        queries，keys，values的形状:
            (batch_size，查询或者“键－值”对的个数，num_hiddens)
        valid_lens　的形状:
            (batch_size，)或(batch_size，查询的个数)
            
        经过变换后，输出的queries，keys，values　的形状:
            (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        
        queries.shape: torch.Size([2, 4, 100])
        keys.shape: torch.Size([2, 6, 100])
        values.shape: torch.Size([2, 6, 100])
        '''
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        '''
        queries.shape: torch.Size([10, 4, 20])
        keys.shape: torch.Size([10, 6, 20])
        values.shape: torch.Size([10, 6, 20])
        '''
        
        '''
        2. 如果有有效长度，则重复这些长度值，使其适应多头计算。
        valid_lens: tensor([3, 2])
        '''
        if valid_lens is not None:
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0
            )
            
        '''
        valid_lens: tensor([3, 3, 3, 3, 3, 2, 2, 2, 2, 2])
        
        3. 通过注意力机制计算输出。
        '''
        output = self.attention(queries, keys, values, valid_lens)
        
        '''
        output的形状:(batch_size*num_heads，查询的个数，num_hiddens/num_heads)
        output.shape: torch.Size([10, 4, 20])
        
        4. 使用 `transpose_output` 函数逆转形状变换，将多个头的输出拼接在一起。
        '''
        output_concat = transpose_output(output, self.num_heads)
        '''
        output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat.shape: torch.Size([2, 4, 100])
        
        5. 最后，通过线性变换得到最终的输出。
        
        self.W_o(output_concat).shape: torch.Size([2, 4, 100])
        '''
        return self.W_o(output_concat)

In [3]:
'''
这个函数用于将输入张量变换为适应多头注意力计算的形状。
'''
def transpose_qkv(X, num_heads):

    '''
    X.shape: torch.Size([2, 4, 100])
    num_heads: 5
    X.shape: torch.Size([2, 6, 100])
    num_heads: 5
    X.shape: torch.Size([2, 6, 100])
    num_heads: 5
    
    - `X`：输入张量，形状为 `(batch_size, num_queries/num_kvpairs, num_hiddens)`。
    - 将 `X` 变换为 `(batch_size, num_queries/num_kvpairs, num_heads, num_hiddens/num_heads)`。
    '''
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    
    '''
    X.shape: torch.Size([2, 4, 5, 20])
    X.shape: torch.Size([2, 6, 5, 20])
    X.shape: torch.Size([2, 6, 5, 20])
    - 使用 `permute` 函数将张量的维度重排列为 `(batch_size, num_heads, num_queries/num_kvpairs, num_hiddens/num_heads)`。
    '''
    X = X.permute(0, 2, 1, 3)
    
    '''
    X.shape: torch.Size([2, 5, 4, 20])
    X.shape: torch.Size([2, 5, 6, 20])
    X.shape: torch.Size([2, 5, 6, 20])
    - 最后，重新变换形状为 `(batch_size * num_heads, num_queries/num_kvpairs, num_hiddens/num_heads)`。
    
    X.reshape(-1, X.shape[2], X.shape[3]).shape: torch.Size([10, 4, 20])
    X.reshape(-1, X.shape[2], X.shape[3]).shape: torch.Size([10, 6, 20])
    X.reshape(-1, X.shape[2], X.shape[3]).shape: torch.Size([10, 6, 20])
    '''
    
    return X.reshape(-1, X.shape[2], X.shape[3])

In [4]:
'''
这个函数用于逆转 `transpose_qkv` 函数的操作。

- `X`：输入张量，形状为 `(batch_size * num_heads, num_queries/num_kvpairs, num_hiddens/num_heads)`。
- 将 `X` 变换为 `(batch_size, num_heads, num_queries/num_kvpairs, num_hiddens/num_heads)`。
- 使用 `permute` 函数将张量的维度重排列为 `(batch_size, num_queries/num_kvpairs, num_heads, num_hiddens/num_heads)`。
- 最后，重新变换形状为 `(batch_size, num_queries/num_kvpairs, num_hiddens)`。
'''
def transpose_output(X, num_heads):
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [5]:
'''
- `num_hiddens` 和 `num_heads`：分别表示隐藏层的神经元数量和注意力头的数量。
'''

num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(
    num_hiddens,
    num_hiddens,
    num_hiddens,
    num_hiddens,
    num_heads,
    0.5
)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [6]:
'''
- `batch_size` 和 `num_queries`：批量大小和查询数量。
- `num_kvpairs` 和 `valid_lens`：键-值对的数量和有效长度。
'''
batch_size, num_queries = 2,4
num_kvpairs, valid_lens = 6, torch.tensor([3, 2])
'''
- `X` 和 `Y`：输入的查询、键和值，都是形状为 `(batch_size, num_queries/num_kvpairs, num_hiddens)` 的张量。

测试代码输出的形状为 `(batch_size, num_queries, num_hiddens)`，表示多头注意力机制的输出形状与输入查询的形状一致。
'''
X = torch.ones(
    (batch_size, num_queries, num_hiddens)
)
Y = torch.ones(
    (batch_size, num_kvpairs, num_hiddens)
)
attention(X, Y, Y, valid_lens).shape


self.W_o(output_concat).shape: torch.Size([2, 4, 100])


torch.Size([2, 4, 100])

多头注意力机制通过并行多个注意力头来增强模型的表示能力，能够捕获输入数据的不同方面。上述代码实现了多头注意力机制的核心步骤，包括查询、键和值的线性变换，形状变换，多头注意力计算，以及最终的线性变换输出。